## Prototype 8
----
## New Features
1. Package Structure similar to CoastSat
2. New ROI generation algorithm
3. ROIs will have their percent overlap written to a csv_file
4. Inputs will be saved to a file

In [1]:
# Internal Imports
import warnings
from ipywidgets import HTML
from ipyleaflet import Marker, Popup
warnings.filterwarnings("ignore")

# External Imports
from ipywidgets import Layout
from ipyleaflet import DrawControl, GeoJSON
import leafmap
import os

# Local Imports 
from CoastSeg import download_roi
from CoastSeg import bbox
from CoastSeg import make_overlapping_roi

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.



In [2]:
# Map Variables
# ---------------
center_point = ( 36, -121.5)
zoom = 13
ROI_SIZE = 0.008

map_settings={
"center_point": center_point,
"zoom":zoom,
 "draw_control":False,
 "measure_control":False, 
 "fullscreen_control":False, 
 "attribution_control":True,
 "Layout":Layout(width='100%', height='100px')
}

# CoastSat Download Variables
# ------------------------------
dates = ['2018-12-01', '2019-01-01']
sat_list = ['S2','L7','L8']
# choose Landsat collection 'C01' or 'C02'
collection = 'C01'

pre_process_settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
}

# Filenames to Store Data
# --------------------------
roi_filename = "official_roi.geojson"
csv_filename='overlap.csv'
selected_roi_file="selected_roi.geojson"
shoreline_file=os.getcwd()+os.sep+"third_party_data"+os.sep+"stanford-xv279yj9196-geojson.json"
inputs_filename="inputs.json"

# Create the Map and Draw a Bounding Box

In [3]:
# Empty list to hold all the polygons drawn by the user
shapes_list=[]

# Disable polyline, circle, and rectangle 
m = leafmap.Map(draw_control=map_settings["draw_control"],
                measure_control=map_settings["measure_control"],
                fullscreen_control=map_settings["fullscreen_control"],
                attribution_control=map_settings["attribution_control"],
                center=map_settings["center_point"],
                zoom=map_settings["zoom"],
                layout=map_settings["Layout"])

draw_control = DrawControl()

draw_control.polyline = {}
draw_control.circlemarker = {}
# Custom styles for polygons and rectangles
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "green",
        "color": "green",
        "fillOpacity": 0.2,
        "Opacity": 0.2
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Ops!"
    },
    "allowIntersection": False,
    "transform":True
}

draw_control = DrawControl()
draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "green",
        "color": "green",
        "fillOpacity": 0.1,
        "Opacity": 0.1
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Ops!"
    },
    "allowIntersection": False,
    "transform":True
}

# Each time a polygon is drawn it is appended to the shapeslist which is used to create the bounding box
def handle_draw(target, action, geo_json):
    if draw_control.last_action == 'created'and draw_control.last_draw['geometry']['type']=='Polygon' :
        shapes_list.append( draw_control.last_draw['geometry'])
    if draw_control.last_action == 'deleted':
        shapes_list.pop()

draw_control.on_draw(handle_draw)
m.add_control(draw_control)

m


Map(center=[36, -121.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

# Generate the ROIs in the Bounding Box
---
- Depending on how big the bounding box is it may take some time
- If you get a bouding box too large or too small error re-run the code to generate the bounding box

## Didn't get enough overlapping ROIs
1. Modify the percent overlap allowed parameter to be a smaller value than the default .65
-  Example where percent overlap allowed = ``.55``:
    ``overlap_btw_vectors_df=make_overlapping_roi.min_overlap_btw_vectors(roi_filename,csv_filename,.55)``
2. Comment out line: ``overlap_btw_vectors_df=make_overlapping_roi.min_overlap_btw_vectors(roi_filename,csv_filename,.65)``
to use the default method of generating overlap between ROIs. This may result in excessive overlap between rois and issues with too many rois clustering in one location.

In [4]:
if __name__ == "__main__":
    #     Make sure |your bounding box is within the allowed size
    bbox.validate_bbox_size(shapes_list)
#   dictionary containing geojson coastline
    roi_coastline=bbox.get_coastline(shoreline_file,shapes_list)
#   coastline styled for the map
    coastline_for_map=bbox.get_coastline_for_map(roi_coastline)
    m.add_layer(coastline_for_map)
    
#   Get the rois using the coastline  within th bounding box
    geojson_polygons=make_overlapping_roi.get_ROIs(roi_coastline,roi_filename,csv_filename)
#   Minimize the overlap between all the vectors. No ROIs overlap will exceed 65%
#   If the ROIs do not overlap each other enough you can:
#   1. Change the percent overlap allowed from .65 to a lower value
#   2. Comment out the line calling min_overlap_btw_vectors
    overlap_btw_vectors_df=make_overlapping_roi.min_overlap_btw_vectors(roi_filename,csv_filename,overlap_percent=.65)

Calculating Overlap:   0%|          | 0/1 [00:00<?, ?it/s]

Removing ROI with Excessive Overlap:   0%|          | 0/5 [00:00<?, ?it/s]

# Click the ROIs you want to download
---
- If the bounding box is in the way of selecting the roi. Click the trash can icon on the left side menu, then click the bounding box, then click save. This will delete the bounding box and allow you to click an ROI.

In [5]:
# Read the geojson for all the ROIs generated
data=download_roi.read_geojson_file(roi_filename)

# GLOBAL VARIABLES used by the map
# --------------------
selected_set = set()
selected_layer = None
# ------------------------

# Add style to each feature in the geojson
for feature in data["features"]:
    feature["properties"]["style"] = {
        "color": "grey",
        "weight": 1,
         "fillColor": "grey",
        "fillOpacity": 0.2,
     }

def convert_selected_set_to_geojson(selected_set):
#     Create a geojson feature collection with no features
    geojson = {"type": "FeatureCollection", "features": []}
# Iterate through all the features in the geojson data and check if its name is in the selected_set
# if its name is then add it as a geojson feature
    geojson["features"] = [
        feature
        for feature in data["features"]
        if feature["properties"]["id"] in selected_set
    ]
 #     change the style for the selected features
    for feature in data["features"]:
        feature["properties"]["style"] = {
            "color": "blue",
            "weight": 2,
             "fillColor": "grey",
             "fillOpacity": 0.2,
          }
    return geojson

# If  properties is none is means there is no name and thus nothing to add
# This function allows for the feature to be removed from the selected portion if it is clicked again after being selected
def selected_onclick_handler(event=None, id=None, properties=None, **args):
    global selected_layer
    if properties is None:
        return
    cid = properties["id"]
    selected_set.remove(cid)
#     If the selected_layer is included then remove it
    if selected_layer is not None:
         m.remove_layer(selected_layer)
    selected_layer = GeoJSON(
        data=convert_selected_set_to_geojson(selected_set),
        name="Selected ROIs",
        hover_style={"fillColor": "blue"},
    )
    selected_layer.on_click(selected_onclick_handler)
    m.add_layer(selected_layer)

# 
def geojson_onclick_handler(event=None, id=None, properties=None, **args):
    """"When a click is registered checks if the id """
    global selected_layer
#     Custom properties associated with geojson
    if properties is None:
           return
    cid = properties["id"]
#     Add the id to the selected_set (unordered, no duplicates, no index)
    selected_set.add(cid)
#     Remove the previously selected layer from map layer
    if selected_layer is not None:
        m.remove_layer(selected_layer)
# Create a new layer from the selected geojson on the map 
# call convert_selected_set_to_geojson() to style the selected geojson
    selected_layer = GeoJSON(
        data=convert_selected_set_to_geojson(selected_set),
        name="Selected ROIs",
        hover_style={"fillColor": "blue"},
    )
# Add an on_click handler to the selected_layer
    selected_layer.on_click(selected_onclick_handler)
    m.add_layer(selected_layer)


geojson_layer = GeoJSON(
    data=data, name="ROIs", hover_style={"fillColor": "red"}
    )
geojson_layer.on_click(geojson_onclick_handler)
m.add_layer(geojson_layer)
m


Map(bottom=103327.0, center=[35.90351225650339, -121.10435485839845], controls=(ZoomControl(options=['position…

## Get the ROIs selected by the user
- Make sure you click at least one roi before running the following code
1. Get the IDs of the ROI clicked by the user
2. Download the data associated with the ROIs using CoastSat
    - inputs_file is the json file where the input data for download imagery with CoastSat will be written to

In [6]:
selected_ROI=download_roi.save_roi(roi_filename, selected_roi_file, selected_set)
selected_ROI

{'features': [{"geometry": {"coordinates": [[[-121.530475, 35.985446], [-121.530475, 36.025446], [-121.490475, 36.025446], [-121.490475, 35.985446], [-121.530475, 35.985446]]], "type": "Polygon"}, "properties": {"id": 2}, "type": "Feature"},
  {"geometry": {"coordinates": [[[-121.514725, 35.966677], [-121.514725, 36.006677], [-121.474725, 36.006677], [-121.474725, 35.966677], [-121.514725, 35.966677]]], "type": "Polygon"}, "properties": {"id": 4}, "type": "Feature"}]}

In [7]:
download_roi.download_imagery(selected_ROI,pre_process_settings,dates,sat_list,collection,inputs_filename=inputs_filename)

Number of images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     S2: 13 images
     L7: 4 images
     L8: 4 images
  Total to download: 21 images
- In Landsat Tier 2 (not suitable for time-series analysis):
     L7: 0 images
     L8: 2 images
  Total Tier 2: 2 images
Number of images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     S2: 13 images
     L7: 4 images
     L8: 4 images
  Total to download: 21 images
- In Landsat Tier 2 (not suitable for time-series analysis):
     L7: 0 images
     L8: 2 images
  Total Tier 2: 2 images
Images available: 
 [{'polygon': [[[-121.530475, 35.985446], [-121.490475, 35.985446], [-121.490475, 36.025446], [-121.530475, 36.025446], [-121.530475, 35.985446]]], 'dates': ['2018-12-01', '2019-01-01'], 'sat_list': ['S2', 'L7', 'L8'], 'sitename': 'ID02022-06-13__17_hr_20_min', 'filepath': 'c:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\data', 'landsat


inputs:  {'polygon': [[[-121.530475, 35.985446], [-121.490475, 35.985446], [-121.490475, 36.025446], [-121.530475, 36.025446], [-121.530475, 35.985446]]], 'dates': ['2018-12-01', '2019-01-01'], 'sat_list': ['S2', 'L7', 'L8'], 'sitename': 'ID02022-06-13__17_hr_20_min', 'filepath': 'c:\\1_USGS\\CoastSeg\\repos\\2_CoastSeg\\CoastSeg_fork\\Seg2Map\\data', 'landsat_collection': 'C01'} 

Number of images available between 2018-12-01 and 2019-01-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
     S2: 13 images
     L7: 4 images
     L8: 4 images
  Total to download: 21 images
- In Landsat Tier 2 (not suitable for time-series analysis):
     L7: 0 images
     L8: 2 images
  Total Tier 2: 2 images

S2: 13 images
100%
L7: 4 images
100%
L8: 4 images
100%

 Called merge_overlapping_images

0 out of 13 Sentinel-2 images were merged (overlapping or duplicate)
Satellite images saved as .jpg in c:\1_USGS\CoastSeg\repos\2_CoastSeg\CoastSeg_fork\Seg2Map\data\ID02022-06-13__17_hr_20_min\jpg_files\prepro